# Tarea 2 - Red Neuronal con Elephas (100% Big Data)

## 1. Importar Librerías

In [ ]:
# ============================= LIBRERÍAS =============================
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración de Spark/Hadoop
os.environ["HADOOP_HOME"] = "C:\\hadoop"

# PySpark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import col, rand
from pyspark.sql.types import DoubleType, StructType, StructField

# Keras/TensorFlow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Elephas - Deep Learning sobre Spark
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

print("✓ Librerías importadas correctamente")

## 2. Configurar Spark Session

In [ ]:
# Crear SparkSession
spark = SparkSession.builder \
    .appName("RedNeuronal_Elephas") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

print(f"✓ Spark Session creada")
print(f"  Versión Spark: {spark.version}")
print(f"  Master: {sc.master}")

## 3. Generar Dataset Sintético (Big Data)
Generamos 100,000 registros con 10 features para demostrar capacidades de Big Data

In [ ]:
# Parámetros del dataset
NUM_RECORDS = 100000  # 100k registros para simular big data
NUM_FEATURES = 10     # 10 características de entrada

# Generar datos distribuidos con Spark
# Creamos un DataFrame con features aleatorias y target binario
data_rdd = sc.parallelize(range(NUM_RECORDS), numSlices=8)

def generate_row(idx):
    """Genera una fila con features aleatorias y target"""
    import random
    random.seed(idx)  # Seed para reproducibilidad
    
    # Generar 10 features aleatorias entre 0 y 1
    features = [random.random() for _ in range(NUM_FEATURES)]
    
    # Target: clasificación binaria basada en suma de features
    # Si suma > 5, target=1, sino target=0
    target = 1.0 if sum(features) > 5.0 else 0.0
    
    return tuple(features + [target])

# Generar datos
generated_rdd = data_rdd.map(generate_row)

# Crear DataFrame
columns = [f"feature_{i}" for i in range(NUM_FEATURES)] + ["target"]
df = generated_rdd.toDF(columns)

print(f"✓ Dataset generado: {df.count():,} registros x {NUM_FEATURES} features")
print(f"\nPrimeras 5 filas:")
df.show(5, truncate=False)

## 4. Preprocesamiento de Datos con Spark ML

In [ ]:
# Ensamblar features en un vector
feature_columns = [f"feature_{i}" for i in range(NUM_FEATURES)]
assembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="features_raw"
)

df_assembled = assembler.transform(df)

# Normalizar features con StandardScaler de Spark ML
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withStd=True,
    withMean=True
)

scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

# Seleccionar solo features y target
df_final = df_scaled.select("features", "target")

print("✓ Datos preprocesados y normalizados")
df_final.show(5, truncate=False)

## 5. Dividir Datos en Train/Test (80/20)

In [ ]:
# Split 80% train, 20% test
train_df, test_df = df_final.randomSplit([0.8, 0.2], seed=42)

train_df = train_df.cache()
test_df = test_df.cache()

train_count = train_df.count()
test_count = test_df.count()

print(f"✓ Datos divididos:")
print(f"  Train: {train_count:,} registros")
print(f"  Test:  {test_count:,} registros")

## 6. Definir Arquitectura de Red Neuronal
### Especificaciones:
- **Capa oculta**: 4 neuronas, activación sigmoid
- **Capa salida**: 1 neurona, activación sigmoid
- **Función de pérdida**: MAE (Mean Absolute Error)
- **Optimizador**: Adam

In [ ]:
def create_model():
    """Crea el modelo de red neuronal según especificaciones"""
    model = Sequential([
        # Capa oculta: 4 neuronas, activación sigmoid
        Dense(
            units=4,
            activation='sigmoid',
            input_shape=(NUM_FEATURES,),
            name='capa_oculta'
        ),
        
        # Capa de salida: 1 neurona, activación sigmoid
        Dense(
            units=1,
            activation='sigmoid',
            name='capa_salida'
        )
    ])
    
    # Compilar con MAE como función de pérdida
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_absolute_error',  # MAE
        metrics=['mae', 'accuracy']
    )
    
    return model

# Crear y mostrar el modelo
model = create_model()
print("✓ Modelo creado")
print("\nArquitectura del modelo:")
model.summary()

## 7. Preparar RDD para Elephas
Convertimos el DataFrame de Spark a formato RDD compatible con Elephas

In [ ]:
def spark_df_to_rdd(df):
    """Convierte Spark DataFrame a RDD en formato (features_array, label)"""
    def row_to_tuple(row):
        # Convertir DenseVector a lista
        features_list = row.features.toArray().tolist()
        label = float(row.target)
        return (features_list, label)
    
    return df.rdd.map(row_to_tuple)

# Convertir train y test a RDD
train_rdd = spark_df_to_rdd(train_df)
test_rdd = spark_df_to_rdd(test_df)

print("✓ DataFrames convertidos a RDD")
print(f"  Train RDD: {train_rdd.count():,} registros")
print(f"  Test RDD: {test_rdd.count():,} registros")
print(f"\nEjemplo de dato en RDD:")
print(train_rdd.take(1)[0])

## 8. Entrenar Modelo con Elephas (Distribuido)
### Parámetros:
- **Minibatch size**: 1000 (según especificación)
- **Epochs**: 5
- **Mode**: 'asynchronous' (entrenamiento asíncrono distribuido)

In [ ]:
# Configuración de Elephas
BATCH_SIZE = 1000  # Minibatch de 1000 según especificación
EPOCHS = 5

# Crear SparkModel de Elephas
spark_model = SparkModel(
    model=model,
    frequency='epoch',      # Sincronización por epoch
    mode='asynchronous',    # Modo asíncrono para mejor performance
    num_workers=4           # Número de workers (ajustar según tu sistema)
)

print("✓ SparkModel de Elephas creado")
print(f"\nIniciando entrenamiento distribuido...")
print(f"  Batch size: {BATCH_SIZE:,}")
print(f"  Epochs: {EPOCHS}")
print(f"  Mode: asynchronous")
print(f"  Registros de entrenamiento: {train_rdd.count():,}\n")

# Entrenar el modelo de forma distribuida
spark_model.fit(
    train_rdd,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_split=0.1
)

print("\n✓ Entrenamiento completado")

## 9. Evaluar Modelo en Test Set

In [ ]:
# Obtener predicciones en test set
predictions_rdd = spark_model.predict(test_rdd)

# Calcular métricas
def calculate_metrics(predictions_rdd, test_rdd):
    """Calcula MAE y Accuracy"""
    # Combinar predicciones con valores reales
    predictions_list = predictions_rdd.collect()
    actuals_list = test_rdd.map(lambda x: x[1]).collect()
    
    # Calcular MAE
    mae = sum(abs(p - a) for p, a in zip(predictions_list, actuals_list)) / len(predictions_list)
    
    # Calcular Accuracy (clasificación binaria con threshold 0.5)
    correct = sum(1 for p, a in zip(predictions_list, actuals_list) 
                  if (p >= 0.5 and a == 1.0) or (p < 0.5 and a == 0.0))
    accuracy = correct / len(predictions_list)
    
    return mae, accuracy

mae, accuracy = calculate_metrics(predictions_rdd, test_rdd)

print("="*60)
print("RESULTADOS DE EVALUACIÓN")
print("="*60)
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Registros evaluados: {test_rdd.count():,}")
print("="*60)

## 10. Mostrar Ejemplos de Predicciones

In [ ]:
# Tomar 10 ejemplos de predicciones
sample_test = test_rdd.take(10)
sample_predictions = predictions_rdd.take(10)

print("\n" + "="*80)
print("EJEMPLOS DE PREDICCIONES")
print("="*80)
print(f"{'#':<5} {'Predicción':<15} {'Real':<10} {'Error Absoluto':<15} {'Correcto':<10}")
print("-"*80)

for i, (test_sample, pred) in enumerate(zip(sample_test, sample_predictions), 1):
    actual = test_sample[1]
    pred_class = 1 if pred >= 0.5 else 0
    actual_class = int(actual)
    error = abs(pred - actual)
    correct = "✓" if pred_class == actual_class else "✗"
    
    print(f"{i:<5} {pred:.4f} ({pred_class})  {actual:.1f} ({actual_class})  "
          f"{error:>8.4f}       {correct:^10}")

print("="*80)

## 11. Resumen Final

In [ ]:
print("\n" + "="*80)
print("RESUMEN DE IMPLEMENTACIÓN")
print("="*80)
print("\n✓ ARQUITECTURA:")
print(f"  - Capa oculta: 4 neuronas, activación sigmoid")
print(f"  - Capa salida: 1 neurona, activación sigmoid")
print(f"  - Total parámetros: {model.count_params():,}")

print("\n✓ CONFIGURACIÓN:")
print(f"  - Función de pérdida: MAE (Mean Absolute Error)")
print(f"  - Optimizador: Adam")
print(f"  - Minibatch size: {BATCH_SIZE:,}")
print(f"  - Epochs: {EPOCHS}")

print("\n✓ DATOS:")
print(f"  - Total registros: {NUM_RECORDS:,}")
print(f"  - Features: {NUM_FEATURES}")
print(f"  - Train: {train_count:,} registros")
print(f"  - Test: {test_count:,} registros")

print("\n✓ RESULTADOS:")
print(f"  - MAE en Test: {mae:.4f}")
print(f"  - Accuracy en Test: {accuracy:.4f} ({accuracy*100:.2f}%)")

print("\n✓ TECNOLOGÍAS:")
print(f"  - PySpark (procesamiento distribuido)")
print(f"  - Elephas (deep learning distribuido)")
print(f"  - Keras/TensorFlow (modelo de red neuronal)")
print(f"  - 100% Big Data pipeline")
print("="*80)

## 12. Limpiar Recursos

In [ ]:
# Unpersist cached DataFrames
train_df.unpersist()
test_df.unpersist()

# Detener Spark
spark.stop()
print("✓ Recursos liberados y Spark detenido")